In [ ]:
## This file implements neural networks and logistic regression on p002ypresabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## To deal with overfitting problem, we include both dropout and regularizer when set up layers in neural networks.
## For fully-connected neural networks, the accuracy is 100% for combination data, and 99.32% for over-sampling data.
## For logistic regression, the accuracy is 99.3% for combination data, and 100% for over-sampling data.
## Since the accuracy scores are pretty high in logistic regression, we further construct random forest models, which 
## are relatively less likely to bring overfitting compared to decision tree.
## For random forest, the accuracy is 99.3% for combination data, and 100% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 99.6% for combination data, and 97.4% for over-sampling.

In [1]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p002ypresabs_quant.csv')
df.shape

(255, 1759)

In [2]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [3]:
df['pheno']

0      0.194875
1      0.265250
2      0.440625
3      0.175500
4      0.173625
5      0.270375
6      0.171000
7      0.182500
8      0.278875
9      0.174375
10     0.176250
11     0.411125
12     0.178250
13     0.191500
14     0.164875
15     0.181875
16     0.394125
17     0.364875
18     0.409875
19     0.191875
20     0.236125
21     0.277000
22     0.169875
23     0.171625
24     0.523625
25     0.169500
26     0.381000
27     0.170000
28     0.160750
29     0.389750
         ...   
225    0.230500
226    0.279800
227    0.319500
228    0.424100
229    0.460500
230    0.304750
231    0.193875
232    0.243000
233    0.195750
234    0.184750
235    0.329125
236    0.160000
237    0.157500
238    0.176375
239    0.326750
240    0.355750
241    0.281375
242    0.181875
243    0.174125
244    0.359250
245    0.161375
246    0.185875
247    0.172750
248    0.167750
249    0.179875
250    0.365375
251    0.223500
252    0.170750
253    0.261125
254    0.231000
Name: pheno, Length: 255

In [4]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [5]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [6]:
df['pheno']
df['pheno'].value_counts()

0    244
1     11
Name: pheno, dtype: int64

In [7]:
df.shape

(255, 1759)

In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 1758)

In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 1757) (255,)


In [11]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 235), (1, 241)]


In [12]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 244), (1, 244)]


In [13]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [18]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [19]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 333 samples, validate on 143 samples
Epoch 1/100
333/333 [==============================] - 0s 788us/step - loss: 2.0204 - accuracy: 0.7207 - val_loss: 0.6234 - val_accuracy: 0.8601
Epoch 2/100
333/333 [==============================] - 0s 154us/step - loss: 2.0650 - accuracy: 0.8078 - val_loss: 0.5509 - val_accuracy: 0.8182
Epoch 3/100
333/333 [==============================] - 0s 176us/step - loss: 1.5537 - accuracy: 0.8468 - val_loss: 0.5270 - val_accuracy: 0.8182
Epoch 4/100
333/333 [==============================] - 0s 148us/step - loss: 1.8920 - accuracy: 0.8649 - val_loss: 0.4832 - val_accuracy: 0.8671
Epoch 5/100
333/333 [==============================] - 0s 142us/step - loss: 1.8230 - accuracy: 0.8589 - val_loss: 0.4564 - val_accuracy: 0.9231
Epoch 6/100
333/333 [==============================] - 0s 300us/step - loss: 1.9959 - accuracy: 0.8498 - val_loss: 0.4770 - val_accuracy: 0.8671
Epoch 7/100
333/333 [==============================] - 0s 182us/step - loss: 2.3043 

Epoch 57/100
333/333 [==============================] - 0s 163us/step - loss: 1.5118 - accuracy: 0.9039 - val_loss: 0.2131 - val_accuracy: 1.0000
Epoch 58/100
333/333 [==============================] - 0s 125us/step - loss: 1.6023 - accuracy: 0.8979 - val_loss: 0.2130 - val_accuracy: 1.0000
Epoch 59/100
333/333 [==============================] - 0s 183us/step - loss: 1.5501 - accuracy: 0.9039 - val_loss: 0.2155 - val_accuracy: 1.0000
Epoch 60/100
333/333 [==============================] - 0s 118us/step - loss: 1.5019 - accuracy: 0.9069 - val_loss: 0.2104 - val_accuracy: 1.0000
Epoch 61/100
333/333 [==============================] - 0s 100us/step - loss: 1.7349 - accuracy: 0.8889 - val_loss: 0.2121 - val_accuracy: 1.0000
Epoch 62/100
333/333 [==============================] - 0s 105us/step - loss: 1.4567 - accuracy: 0.9099 - val_loss: 0.2135 - val_accuracy: 1.0000
Epoch 63/100
333/333 [==============================] - 0s 143us/step - loss: 1.1310 - accuracy: 0.9309 - val_loss: 0.2059 -

In [21]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

143/143 [==============================] - 0s 87us/step
combination test accuracy: 100.00%


In [22]:
###### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [23]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 341 samples, validate on 147 samples
Epoch 1/100
341/341 [==============================] - 0s 640us/step - loss: 2.5008 - accuracy: 0.6188 - val_loss: 0.6546 - val_accuracy: 0.7619
Epoch 2/100
341/341 [==============================] - 0s 122us/step - loss: 1.9459 - accuracy: 0.7126 - val_loss: 0.6206 - val_accuracy: 0.7755
Epoch 3/100
341/341 [==============================] - 0s 127us/step - loss: 2.1338 - accuracy: 0.7009 - val_loss: 0.6132 - val_accuracy: 0.7755
Epoch 4/100
341/341 [==============================] - 0s 126us/step - loss: 2.3887 - accuracy: 0.7097 - val_loss: 0.6141 - val_accuracy: 0.7755
Epoch 5/100
341/341 [==============================] - 0s 127us/step - loss: 1.9817 - accuracy: 0.7507 - val_loss: 0.5801 - val_accuracy: 0.7619
Epoch 6/100
341/341 [==============================] - 0s 160us/step - loss: 1.4946 - accuracy: 0.7947 - val_loss: 0.5751 - val_accuracy: 0.7619
Epoch 7/100
341/341 [==============================] - 0s 126us/step - loss: 1.9504 

341/341 [==============================] - 0s 154us/step - loss: 1.7729 - accuracy: 0.8886 - val_loss: 0.3049 - val_accuracy: 0.9932
Epoch 57/100
341/341 [==============================] - 0s 125us/step - loss: 1.8177 - accuracy: 0.8768 - val_loss: 0.3636 - val_accuracy: 0.8844
Epoch 58/100
341/341 [==============================] - 0s 136us/step - loss: 1.6396 - accuracy: 0.8944 - val_loss: 0.2977 - val_accuracy: 0.9932
Epoch 59/100
341/341 [==============================] - 0s 118us/step - loss: 1.8126 - accuracy: 0.8798 - val_loss: 0.3020 - val_accuracy: 0.9932
Epoch 60/100
341/341 [==============================] - 0s 109us/step - loss: 1.8629 - accuracy: 0.8739 - val_loss: 0.3085 - val_accuracy: 1.0000
Epoch 61/100
341/341 [==============================] - 0s 106us/step - loss: 1.5919 - accuracy: 0.8944 - val_loss: 0.3021 - val_accuracy: 0.9932
Epoch 62/100
341/341 [==============================] - 0s 286us/step - loss: 1.3654 - accuracy: 0.9091 - val_loss: 0.3170 - val_accuracy

In [25]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

147/147 [==============================] - 0s 109us/step
over-sampling test accuracy: 99.32%


In [ ]:
############ Logistic Regression ############

In [90]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [91]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [99]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 99.30%


In [96]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [98]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 100.00%


In [100]:
############## Random Forest ##############

[1.        1.        0.9787234 1.        1.       ]
0.9957446808510639
0.008510638297872353


In [104]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


In [112]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.30%


In [113]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [114]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

combination test accuracy: 100.00%


In [ ]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [116]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.        1.        0.9787234 1.        1.       ]
0.9957446808510639


In [117]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.97916667 1.         0.97916667 0.97916667 0.93478261]
0.9744565217391303
